## Прогноз виручки (three-phase linear)

Щомісячний прогноз виручки по категоріях із повторним використанням пайплайну.

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

from three_phase_linear import ForecastConfig, run_three_phase_forecast

DATA_PATH = Path('forecast_revenue_dataset.csv')
OUTPUT_PATH = Path('money_three_phase_forecast.csv')
GROUP_COLS = ['category_id']
TARGET_COLUMN = 'revenue'
REGRESSORS = ['is_sale_prohibition', 'cos_month', 'sin_month', 'cos_quarter', 'sin_quarter', 'unique_brand_count']

In [2]:
df = pd.read_csv(DATA_PATH)
df['date'] = pd.to_datetime(df['date'])
for col in REGRESSORS:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df = df.sort_values(GROUP_COLS + ['date']).reset_index(drop=True)

future_counts = df[df[TARGET_COLUMN].isna()].groupby(GROUP_COLS).size()
forecast_horizon = int(future_counts.max()) if not future_counts.empty else 12
if forecast_horizon <= 0:
    forecast_horizon = 12

print(f'Горизонт прогнозу: {forecast_horizon} періодів')

Горизонт прогнозу: 72 періодів


In [3]:
input_cols = ['date', *GROUP_COLS, TARGET_COLUMN, *REGRESSORS]
input_cols = list(dict.fromkeys(input_cols))
config = ForecastConfig(
    time_col='date',
    target_col=TARGET_COLUMN,
    group_cols=GROUP_COLS,
    freq='MS',
    forecast_horizon=forecast_horizon,
    seasonal_periods=12,
    min_history=24,
    lags=(1, 2, 3, 6, 12, 18, 24),
    rolling_windows=(3, 6, 12, 24),
    additional_regressors=REGRESSORS,
    random_search_iterations=10,
    n_splits=4,
    random_state=46,
)

preds, summaries = run_three_phase_forecast(df[input_cols].copy(), config)
preds = preds.rename(columns={
    'prediction': 'revenue_forecast',
    f'{TARGET_COLUMN}_holtwinters': 'revenue_baseline',
})
summary_report = pd.DataFrame({
    'group_key': [s.group_key[0] for s in summaries],
    'train_rows': [s.train_rows for s in summaries],
    'cv_mae': [s.best_score for s in summaries],
    'skipped_reason': [s.skipped_reason for s in summaries],
})
summary_report.head()

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


...

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


,group_key,train_rows,cv_mae,skipped_reason
0,1,408,1.362843e+07,None
1,2,408,1.295078e+07,None
2,3,408,4.757858e+07,None
3,4,408,1.111939e+07,None
4,5,408,7.521579e+06,None


In [4]:
merge_cols = [*GROUP_COLS, 'date']
result_df = df.copy()
result_df['is_forecast_period'] = result_df[TARGET_COLUMN].isna()
result_df = result_df.merge(preds[merge_cols + ['revenue_forecast']], on=merge_cols, how='left')
result_df['revenue'] = result_df['revenue'].astype(float)
result_df['revenue'] = result_df['revenue'].fillna(result_df['revenue_forecast'])

output_columns = ['date', 'category_id', 'category_title', 'revenue']
final_output = result_df.loc[result_df['is_forecast_period'], output_columns]
final_output = final_output.groupby(['date', 'category_id', 'category_title'], as_index=False)['revenue'].sum()
final_output = final_output.sort_values(['category_id', 'date']).reset_index(drop=True)
final_output.to_csv(OUTPUT_PATH, index=False)

final_output.tail()

,date,category_id,category_title,revenue
85,2026-04-01,5,Vodka,1.412862e+09
86,2026-05-01,5,Vodka,1.431305e+09
87,2026-06-01,5,Vodka,1.534269e+09
88,2026-07-01,5,Vodka,1.479408e+09
89,2026-08-01,5,Vodka,1.494589e+09


In [5]:
summary_report

,group_key,train_rows,cv_mae,skipped_reason
0,1,408,1.362843e+07,None
1,2,408,1.295078e+07,None
2,3,408,4.757858e+07,None
3,4,408,1.111939e+07,None
4,5,408,7.521579e+06,None
